In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Titanic

/content/drive/MyDrive/Titanic


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.impute import SimpleImputer

In [ ]:
train_data = pd.read_csv('train.csv')
y = train_data.Survived
needed_features = ['Pclass','Sex','Age','SibSp','Parch','Cabin','Embarked']
X = train_data[needed_features]
X.head()

,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked
0,3,male,22.0,1,0,NaN,S
1,1,female,38.0,1,0,C85,C
2,3,female,26.0,0,0,NaN,S
3,1,female,35.0,1,0,C123,S
4,3,male,35.0,0,0,NaN,S


In [ ]:
cols_with_missing = [col for col in X.columns
                     if X[col].isnull().any()]
cols_with_missing

['Age', 'Cabin', 'Embarked']

In [ ]:
X_train = pd.get_dummies(X)
cols_with_missing1 = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
for cols in cols_with_missing1:
  X_train[cols].fillna(X_train[cols].median(),inplace=True)
cols_with_missing1 = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
X_train.head()

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,38.0,1,0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,26.0,0,0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1,35.0,1,0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,3,35.0,0,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
test_data = pd.read_csv('test.csv')
X_test = test_data[needed_features]
X_test = pd.get_dummies(X_test)
cols_with_missing_in_test = [col for col in X_test.columns if X_test[col].isnull().any()]
# print(cols_with_missing_in_test)
missed_in_test = [col for col in X_test.columns if col not in X_train.columns]
for col in missed_in_test:
  X_train[col] = False
X_test['Age'].fillna(X_test["Age"].median(),inplace=True)
missed_in_test = [col for col in X_train.columns if col not in X_test.columns]
for col in missed_in_test:
  X_test = pd.concat([X_test,X_train[col]],axis=1)
cols_in_common = list(X_train.columns)
X_test = X_test[cols_in_common]
X_test.head()

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,...,Cabin_D38,Cabin_D40,Cabin_D43,Cabin_E39 E41,Cabin_E45,Cabin_E52,Cabin_E60,Cabin_F,Cabin_F E46,Cabin_F E57
0,3.0,34.5,0.0,0.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,3.0,47.0,1.0,0.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2.0,62.0,0.0,0.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3.0,27.0,0.0,0.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3.0,22.0,1.0,1.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
for sets in [X_train,X_test]:
  sets.insert(len(sets.columns),'Family', np.array(sets.SibSp,int)+np.array(sets.Parch,int))
  lonelness = []
  for fam in sets.Family:
    if fam==0:
      lonelness.append(1)
    else:
      lonelness.append(0)
  sets.insert(len(sets.columns),'Is_Lonely',lonelness)

ValueError: cannot insert Is_Lonely, already exists

In [ ]:
X_test.head()

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,...,Cabin_D43,Cabin_E39 E41,Cabin_E45,Cabin_E52,Cabin_E60,Cabin_F,Cabin_F E46,Cabin_F E57,Family,Is_Lonely
0,3.0,34.5,0.0,0.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1
1,3.0,47.0,1.0,0.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,0
2,2.0,62.0,0.0,0.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1
3,3.0,27.0,0.0,0.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1
4,3.0,22.0,1.0,1.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0


In [ ]:
X_train.head()

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,...,Cabin_D43,Cabin_E39 E41,Cabin_E45,Cabin_E52,Cabin_E60,Cabin_F,Cabin_F E46,Cabin_F E57,Family,Is_Lonely
0,3,22.0,1,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,1,0
1,1,38.0,1,0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,0
2,3,26.0,0,0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1
3,1,35.0,1,0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,0
4,3,35.0,0,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train_X,val_X,train_y,val_y = train_test_split(X_train,y,train_size=0.8, test_size=0.2,random_state=42)

In [ ]:
train_X.head()

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,...,Cabin_D43,Cabin_E39 E41,Cabin_E45,Cabin_E52,Cabin_E60,Cabin_F,Cabin_F E46,Cabin_F E57,Family,Is_Lonely
331,1,45.5,0,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1
733,2,23.0,0,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1
382,3,32.0,0,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,0,1
704,3,26.0,1,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,1,0
813,3,6.0,4,2,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,6,0


In [ ]:
model = RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(train_X,train_y)

RandomForestClassifier(random_state=42)

In [ ]:
y_val_predict = model.predict(val_X)

In [ ]:
accuracy_of_it = accuracy_score(y_val_predict,val_y)
print('Accuracy of model without tuning is: ',accuracy_of_it)

Accuracy of model without tuning is:  0.7877094972067039
